# Using the line and memory profilers
## `line_profiler`

In [1]:
import numpy as np

In [2]:
def euclidean_broadcast(x, y):
    diff = x[:, np.newaxis, :] - y[np.newaxis, :, :]
    return (diff * diff).sum(axis=2)

In [3]:
def euclidean_trick(x, y):
    x2 = np.einsum('ij,ij->i', x, x)[:, np.newaxis]
    y2 = np.einsum('ij,ij->i', y, y)[np.newaxis, :]
    xy = np.dot(x, y.T)
    return np.abs(x2 + y2 - 2. * xy)

In [4]:
nsamples = 2000
nfeat = 50

x = 10. * np.random.random([nsamples, nfeat])

%timeit euclidean_broadcast(x, x)
%timeit euclidean_trick(x, x)

1.42 s ± 2 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
84.7 ms ± 750 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [5]:
%load_ext line_profiler

In [6]:
%lprun?

Docstring:
Execute a statement under the line-by-line profiler from the
line_profiler module.

Usage:
%lprun -f func1 -f func2 <statement>

The given statement (which doesn't require quote marks) is run via the
LineProfiler. Profiling is enabled for the functions specified by the -f
options. The statistics will be shown side-by-side with the code through the
pager once the statement has completed.

Options:

-f <function>: LineProfiler only profiles functions and methods it is told
to profile.  This option tells the profiler about these functions. Multiple
-f options may be used. The argument may be any expression that gives
a Python function or method object. However, one must be careful to avoid
spaces that may confuse the option parser.

-m <module>: Get all the functions/methods in a module

One or more -f or -m options are required to get any useful results.

-D <filename>: dump the raw statistics out to a pickle file on disk. The
usual extension for this is ".lprof". These statis

In [7]:
%lprun -f euclidean_trick euclidean_trick(x, x)

Timer unit: 1e-09 s

Total time: 0.0826756 s
File: /scr/13306566.1.all.q/ipykernel_13955/3465809475.py
Function: euclidean_trick at line 1

Line #      Hits         Time  Per Hit   % Time  Line Contents
     1                                           def euclidean_trick(x, y):
     2         1     505563.0 505563.0      0.6      x2 = np.einsum('ij,ij->i', x, x)[:, np.newaxis]
     3         1     118092.0 118092.0      0.1      y2 = np.einsum('ij,ij->i', y, y)[np.newaxis, :]
     4         1   27113912.0 27113912.0     32.8      xy = np.dot(x, y.T)
     5         1   54938024.0 54938024.0     66.5      return np.abs(x2 + y2 - 2. * xy)

In [8]:
%lprun -f euclidean_broadcast euclidean_broadcast(x,x)

Timer unit: 1e-09 s

Total time: 1.4136 s
File: /scr/13306566.1.all.q/ipykernel_13955/709754415.py
Function: euclidean_broadcast at line 1

Line #      Hits         Time  Per Hit   % Time  Line Contents
     1                                           def euclidean_broadcast(x, y):
     2         1  656892146.0 656892146.0     46.5      diff = x[:, np.newaxis, :] - y[np.newaxis, :, :]
     3         1  756710921.0 756710921.0     53.5      return (diff * diff).sum(axis=2)

## cProfiler

In [9]:
%prun euclidean_trick(x, x)

         25 function calls in 0.083 seconds

   Ordered by: internal time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.055    0.055    0.082    0.082 3465809475.py:1(euclidean_trick)
        3    0.027    0.009    0.027    0.009 {built-in method numpy.core._multiarray_umath.implement_array_function}
        1    0.001    0.001    0.083    0.083 <string>:1(<module>)
        2    0.001    0.000    0.001    0.000 {built-in method numpy.core._multiarray_umath.c_einsum}
        1    0.000    0.000    0.083    0.083 {built-in method builtins.exec}
        2    0.000    0.000    0.001    0.000 <__array_function__ internals>:177(einsum)
        1    0.000    0.000    0.027    0.027 <__array_function__ internals>:177(dot)
        2    0.000    0.000    0.001    0.000 einsumfunc.py:1009(einsum)
       10    0.000    0.000    0.000    0.000 einsumfunc.py:1001(_einsum_dispatcher)
        1    0.000    0.000    0.000    0.000 {method 'disable' of '_lsprof.